In [1]:
import os
# Лечим ошибку ядер (на всякий случай)
os.environ['LOKY_MAX_CPU_COUNT'] = '4'

import pandas as pd
import pickle
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.ensemble import HistGradientBoostingClassifier 

# --- 1. СБОРКА ДАТАСЕТА (MERGE) ---
print("⏳ Загружаем умные признаки (PRO)...")
df_features = pd.read_parquet('features_pro.parquet')

print("⏳ Загружаем ответы (Target)...")
# Проверь имя файла! Обычно это train_target.csv
df_target = pd.read_csv('train_target.csv')

print("🔗 Объединяем...")
df = df_features.merge(df_target, on='id')

print(f"✅ Итоговый датасет: {df.shape}")

# Чистим память (удаляем лишние переменные)
del df_features, df_target

# --- 2. ПОДГОТОВКА К БОЮ ---
X = df.drop(columns=['flag', 'id'])
y = df['flag']

# Разделяем (80/20)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print(f"👨‍🏫 Train: {X_train.shape}, Test: {X_test.shape}")

# # --- 3. ОБУЧЕНИЕ (Sklearn PRO) ---
# print("\n⚙️ Запускаем PRO-Pipeline...")

# pipe = Pipeline([
#     ('imputer', SimpleImputer(strategy='median')), 
#     ('scaler', StandardScaler()),                   
#     ('model', HistGradientBoostingClassifier(       
#         max_iter=1000,              # Дадим ему время подумать
#         learning_rate=0.03,
#         max_depth=8,                # Глубина 8 (золотая середина)
#         l2_regularization=3,
#         scoring='roc_auc',
#         random_state=42,
#         verbose=1
#     ))
# ])

# print("🚀 Поехали! Обучение началось...")
# pipe.fit(X_train, y_train)

# # --- 4. ПРОВЕРКА ---
# print("\n🎓 Финальный экзамен...")
# preds_proba = pipe.predict_proba(X_test)[:, 1]
# auc_score = roc_auc_score(y_test, preds_proba)

# print(f"==========================================")
# print(f"🏆 РЕЗУЛЬТАТ (PRO Features): {auc_score:.5f}")
# print(f"==========================================")

# # Сохраняем модель
# with open('model_pro.pkl', 'wb') as file:
#     pickle.dump(pipe, file)
# print("💾 Модель сохранена как model_pro.pkl")

# --- 3. ОБУЧЕНИЕ (Sklearn PRO + BALANCED) ---
print("\n⚙️ Запускаем ФИНАЛЬНЫЙ Pipeline...")

pipe = Pipeline([
    ('imputer', SimpleImputer(strategy='median')), 
    ('scaler', StandardScaler()),                   
    ('model', HistGradientBoostingClassifier(       
        max_iter=1200,              # Чуть больше итераций
        learning_rate=0.03,
        max_depth=8,
        l2_regularization=3,
        class_weight='balanced',    # <--- ВОТ ОНА, КНОПКА "БАБЛО" 
        scoring='roc_auc',
        random_state=42,
        verbose=1
    ))
])

print("🚀 Поехали! Обучение с балансировкой...")
pipe.fit(X_train, y_train)

# --- 4. ПРОВЕРКА ---
print("\n🎓 Финальный экзамен...")
preds_proba = pipe.predict_proba(X_test)[:, 1]
auc_score = roc_auc_score(y_test, preds_proba)

print(f"==========================================")
print(f"🏆 РЕЗУЛЬТАТ (PRO + Balanced): {auc_score:.5f}")
print(f"==========================================")

# Сохраняем модель для сдачи
with open('model_pro.pkl', 'wb') as file:
    pickle.dump(pipe, file)
print("💾 Модель перезаписана!")

⏳ Загружаем умные признаки (PRO)...
⏳ Загружаем ответы (Target)...
🔗 Объединяем...
✅ Итоговый датасет: (3000000, 80)
👨‍🏫 Train: (2400000, 78), Test: (600000, 78)

⚙️ Запускаем ФИНАЛЬНЫЙ Pipeline...
🚀 Поехали! Обучение с балансировкой...
Binning 1.348 GB of training data: 0.911 s
Binning 0.150 GB of validation data: 0.082 s
Fitting gradient boosted rounds:
Fit 634 trees in 234.396 s, (19654 total leaves)
Time spent computing histograms: 167.311s
Time spent finding best splits:  1.076s
Time spent applying splits:      8.888s
Time spent predicting:           1.498s

🎓 Финальный экзамен...
🏆 РЕЗУЛЬТАТ (PRO + Balanced): 0.71968
💾 Модель перезаписана!
